In [1]:
import json
import requests
import datetime
import time
import pandas as pd
from collections import defaultdict
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
SLEEP_DURATION = 7
'''
We need to update the sentiment from today's news articles before we can run the modeling for predicting next day's high.
We use New York Times Developer API to fetch the news articles from the Business section. There is a rate limit on fast we 
can access the data using the free API (10 articles/min).
We need to also deal with the fact the API returns 10 articles at a time; so we need to make multiple calls to finish
processing all the news articles for today.
For each news article's headline and its corresponding lead paragraph, we use VADER sentiment intensity analyzer to 
get its sentiment scores (positive, negative, neutral, compound).
We save the set of average scores across all articles in the day and save the collected sentiment data into a csv file.
'''
def find_news_articles(begindate, nytimes_section):
    base_url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?'
    api_key = '&api-key=INSERT_KEY_HERE'
    facet_str = f'&facet=true&begin_date={begindate}&end_date={begindate}'
    
    page = 0
    count = 0
    
    ret_list = []
    while True:
        url = base_url+nytimes_section+facet_str+f'&page={page}'+api_key
        r = requests.get(url)
        if r.status_code != 200:
            print(r.status_code)
        data = json.loads(r.content)
        time.sleep(SLEEP_DURATION)
        if page == 0:
            tot_articles = data['response']['meta']['hits']
            print(begindate, nytimes_section, 'tot_articles', tot_articles)
        for i, doc in enumerate(data['response']['docs']):
            ret_list.append((doc['headline']['main'], doc['lead_paragraph'], doc['web_url']))
            count += 1
        if count >= tot_articles:
            break
        page += 1
    #print(len(ret_list))
    return ret_list

In [ ]:
BATCH_SIZE = 30

today = datetime.datetime.today()
begin_date = datetime.datetime.strptime('25/03/2007', '%d/%m/%Y')

while begin_date <= today:

    daily_sentiment = defaultdict(defaultdict)
    sid_obj = SentimentIntensityAnalyzer()
    
    i = 0
    while i < BATCH_SIZE:
        days_sentiment_pos, days_sentiment_neg, days_sentiment_neu, days_sentiment_comp = 0, 0, 0, 0
        date_str = str(begin_date.year) + str(begin_date.month).zfill(2) + str(begin_date.day).zfill(2)

        news_desk_str = 'fq=news_desk:("Financial" "Business" "Business Day")'
        section_str = 'fq=section_name:("Your Money" "Business" "Business Day")'

        news_desk_list = find_news_articles(date_str, news_desk_str)
        section_list = find_news_articles(date_str, section_str)

        final_urls = set()
        for news in news_desk_list: # tuple of 3: headline, lead_paragraph, web_url
            if news[2] not in final_urls:
                # print('adding news desk article', news[0])
                final_urls.add(news[2])
                sentiment_dict = sid_obj.polarity_scores(news[0] + news[1])
                days_sentiment_pos += sentiment_dict['pos']
                days_sentiment_neg += sentiment_dict['neg']
                days_sentiment_neu += sentiment_dict['neu']
                days_sentiment_comp += sentiment_dict['compound']
        for news in section_list: # tuple of 3: headline, lead_paragraph, web_url
            if news[2] not in final_urls:
                # print('adding section article', news[0])
                final_urls.add(news[2])
                sentiment_dict = sid_obj.polarity_scores(news[0] + news[1])
                days_sentiment_pos += sentiment_dict['pos']
                days_sentiment_neg += sentiment_dict['neg']
                days_sentiment_neu += sentiment_dict['neu']
                days_sentiment_comp += sentiment_dict['compound']

        num_news_items = len(final_urls)
        if num_news_items > 0:
            daily_sentiment[date_str]['pos'] = days_sentiment_pos/num_news_items
            daily_sentiment[date_str]['neg'] = days_sentiment_neg/num_news_items
            daily_sentiment[date_str]['neu'] = days_sentiment_neu/num_news_items
            daily_sentiment[date_str]['compound'] = days_sentiment_comp/num_news_items
        else:
            daily_sentiment[date_str]['pos'] = 0
            daily_sentiment[date_str]['neg'] = 0
            daily_sentiment[date_str]['neu'] = 0
            daily_sentiment[date_str]['compound'] = 0
        i += 1
        begin_date += datetime.timedelta(days=1)
    df = pd.DataFrame(daily_sentiment).T
    df.to_csv('sentiment_data/' + date_str + '.csv')